# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('task_freq.npz') 

# Access the concatenated features from the .npz file
X_freq = data['X_frequency']

# Close the loaded file
data.close()

#print the merged array
X_freq

array([[[ 2.19931450e-12,  6.36707013e-21,  1.12863886e-22, ...,
          3.84177727e-02,  5.89511357e-02, -4.90758164e-05],
        [ 8.08942099e-13,  1.95776978e-21,  1.79651267e-23, ...,
          1.28670108e+00,  7.65465876e-01, -6.86075951e-06],
        [ 9.36813353e-13,  3.89871747e-21,  2.52142249e-23, ...,
          3.06491400e-01,  4.04279211e-01,  5.97370383e-06],
        ...,
        [ 1.01987363e-12,  3.96770892e-21,  2.94412978e-23, ...,
          6.85096489e-01,  6.04268021e-01,  1.14557782e-05],
        [ 1.02559538e-12,  4.54868682e-21,  3.70363154e-23, ...,
          2.36816655e+00,  1.14725539e+00,  7.30817528e-06],
        [ 1.71736246e-12,  4.19962512e-20,  4.88845537e-23, ...,
          1.61189028e-01,  2.51458784e-01, -1.48651198e-06]],

       [[ 7.06546258e-13,  7.52725095e-21,  1.09906112e-23, ...,
          1.51398843e-01,  2.45259555e-01,  1.35628821e-05],
        [ 9.55642835e-13,  3.01987672e-22,  2.64524742e-23, ...,
          1.49823571e+00,  1.39708834e

In [3]:
data = np.load('task_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('task_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_freq.shape, Y.shape, groups_array.shape)

(38049, 20, 20) (38049,) (38049,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_freq, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_freq[train_index], X_freq[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
134/134 [==============================] - 10s 32ms/step - loss: 0.3110 - accuracy: 0.8730 - val_loss: 0.2765 - val_accuracy: 0.8936
Epoch 2/200
134/134 [==============================] - 3s 21ms/step - loss: 0.2327 - accuracy: 0.9065 - val_loss: 0.2230 - val_accuracy: 0.9114
Epoch 3/200
134/134 [==============================] - 3s 21ms/step - loss: 0.2081 - accuracy: 0.9159 - val_loss: 0.1919 - val_accuracy: 0.9240
Epoch 4/200
134/134 [==============================] - 3s 21ms/step - loss: 0.1928 - accuracy: 0.9232 - val_loss: 0.1852 - val_accuracy: 0.9246
Epoch 5/200
134/134 [==============================] - 3s 21ms/step - loss: 0.1810 - accuracy: 0.9283 - val_loss: 0.1837 - val_accuracy: 0.9254
Epoch 6/200
134/134 [==============================] - 3s 21ms/step - loss: 0.1733 - accuracy: 0.9314 - val_loss: 0.1653 - val_accuracy: 0.9361
Epoch 7/200
134/134 [==============================] - 3s 20ms/step - loss: 0.1628 - accuracy: 0.9361 - val_loss: 0.1662 - val_a

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9703,0.9626,0.9842,0.9733,0.9688,0.9533,0.9842,0.9399,0.9402
1,0.9735,0.9779,0.9737,0.9758,0.9734,0.9731,0.9737,0.9464,0.9464
2,0.9750,0.9730,0.9818,0.9774,0.9743,0.9667,0.9818,0.9495,0.9496
3,0.9735,0.9711,0.9809,0.9760,0.9726,0.9644,0.9809,0.9463,0.9464
4,0.9732,0.9663,0.9857,0.9759,0.9718,0.9580,0.9857,0.9457,0.9460
5,0.9737,0.9672,0.9857,0.9763,0.9724,0.9591,0.9857,0.9468,0.9470
6,0.9803,0.9777,0.9866,0.9822,0.9796,0.9726,0.9866,0.9601,0.9602
7,0.9763,0.9735,0.9838,0.9786,0.9755,0.9673,0.9838,0.9522,0.9522
8,0.9714,0.9693,0.9790,0.9741,0.9705,0.9620,0.9790,0.9421,0.9421
9,0.9745,0.9752,0.9785,0.9769,0.9741,0.9696,0.9785,0.9485,0.9485


In [13]:
metrics_df.round(4).to_csv('FREQ_TASK_CNN_LSTM.csv', index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
result = pd.read_csv("FREQ_TASK_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9703,0.9626,0.9842,0.9733,0.9688,0.9533,0.9842,0.9399,0.9402
1,0.9735,0.9779,0.9737,0.9758,0.9734,0.9731,0.9737,0.9464,0.9464
2,0.9750,0.9730,0.9818,0.9774,0.9743,0.9667,0.9818,0.9495,0.9496
3,0.9735,0.9711,0.9809,0.9760,0.9726,0.9644,0.9809,0.9463,0.9464
4,0.9732,0.9663,0.9857,0.9759,0.9718,0.9580,0.9857,0.9457,0.9460
5,0.9737,0.9672,0.9857,0.9763,0.9724,0.9591,0.9857,0.9468,0.9470
6,0.9803,0.9777,0.9866,0.9822,0.9796,0.9726,0.9866,0.9601,0.9602
7,0.9763,0.9735,0.9838,0.9786,0.9755,0.9673,0.9838,0.9522,0.9522
8,0.9714,0.9693,0.9790,0.9741,0.9705,0.9620,0.9790,0.9421,0.9421
9,0.9745,0.9752,0.9785,0.9769,0.9741,0.9696,0.9785,0.9485,0.9485


In [16]:
(result.mean()*100).round(2)

Accuracy       97.42
Precision      97.14
Recall         98.20
F1             97.66
Sensitivity    97.33
Specificity    96.46
ROC_AUC        98.20
Kappa          94.78
MCC            94.79
dtype: float64

In [19]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])